# Visualing the Attention Maps

This experiment is in visualizing the attention maps in GPT-2. I want to see
- which token the different heads of a single layer pay attention to
- which token the same heads in different layers pay attention to. 

In [6]:
import torch as t
import pandas as pd
from transformers import AutoModel, AutoTokenizer, GPT2Model, GPT2TokenizerFast, GPT2LMHeadModel
from datasets import load_dataset
import altair as alt

device = "cuda" if t.cuda.is_available() else "cpu"
device = "mps" if t.backends.mps.is_available() else "cpu"

In [21]:
model_id = "openai-community/gpt2-medium"
model = AutoModel.from_pretrained(model_id, output_attentions=True)
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [26]:
txt = "The cat ate the mat. And then sat on a rat"
tokens = tokenizer(txt)
inputs = tokenizer(txt, return_tensors='pt').input_ids
tokens, inputs

({'input_ids': [464, 3797, 15063, 262, 2603, 13, 843, 788, 3332, 319, 257, 4227], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]},
 tensor([[  464,  3797, 15063,   262,  2603,    13,   843,   788,  3332,   319,
            257,  4227]]))

In [25]:
attns = model(inputs).attentions
first_layer_attn = attns[0]
first_layer_attn.shape

torch.Size([1, 16, 12, 12])